<a href="https://colab.research.google.com/github/Ahmed-L/CSE499A/blob/main/Dataset_499A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping North South University website

imports

In [ ]:
import os
import torch
from torch.utils.data import Dataset
import pandas as pd
import bs4
from html.parser import HTMLParser
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import numpy as np
import pandas as pd

python's HTMLParser

In [ ]:
from html.entities import name2codepoint

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Start tag:", tag)
        for attr in attrs:
            print("     attr:", attr)

    def handle_endtag(self, tag):
        print("End tag  :", tag)

    def handle_data(self, data):
        print("Data     :", data)

    def handle_comment(self, data):
        print("Comment  :", data)

    def handle_entityref(self, name):
        c = chr(name2codepoint[name])
        print("Named ent:", c)

    def handle_charref(self, name):
        if name.startswith('x'):
            c = chr(int(name[1:], 16))
        else:
            c = chr(int(name))
        print("Num ent  :", c)

    def handle_decl(self, data):
        print("Decl     :", data)

parser = MyHTMLParser()

Demo run Using BS4 

In [ ]:
document = requests.get('http://www.northsouth.edu/about/brief-history.html')
soup = BeautifulSoup(document.text, "html.parser")
print(soup.title.string)
#print(soup.find('h1').text.strip())
print(soup.h2.string)
p = (soup.find_all('p'))

for i in p:
  print(i.string)

divs = soup.find_all('div')

for div in divs:
  try:
    if len(div.string) > 0:
      print(div.string)
  except:
    continue

NSU: A BRIEF HISTORY | North South University
NSU: A BRIEF HISTORY
North South University (NSU), the first private university in Bangladesh, was established in 1992 by the then Foundation for Promotion of Education and Research (FPER), a charitable, non-profit, non-commercial and non-political organization. The FPER later was renamed as the NSU Foundation and is presently called The North South Foundation for Education and Research. The Foundation is comprised of a group of eminent industrialists, prominent patrons of education, notable philanthropists, widely experienced academics and senior civil servants of the country. In the early 1990s, they had a dream to set up a world-class university as a center of excellence in higher education in the private sector. Their dedication, tireless efforts, and hard work paved the way for the approval of the establishment of NSU.
Since, at that time, there was no relevant law in the country to set up and operate a university in the private sector

Fetch all links from the base index page (Does not contain sub-page links)

In [ ]:
index = 'http://www.northsouth.edu/'
document = requests.get(index)
links = []
soup = BeautifulSoup(document.text, "html.parser")

for link in soup.findAll('a'):
  var = link.get('href')
  if(var == '#' or var is None):
    continue
  else:
    if('http' in var):
      #print(var)
      links.append(var)
    else:
      var = index + var
      links.append(var)
      #print(var)


Cleaning, remove PDF links, career, archive

In [ ]:
def format(links):
  prev_length = len(links)
  new_length = 0

  while(prev_length != new_length):
    prev_length = new_length
    for link in links:
      if('pdf' in link):
        links.remove(link)
      elif '.pdf' in link:
        links.remove(link)
      elif ('career' in link):
        links.remove(link)
      elif 'archive' in link:
        links.remove(link)
      elif 'youtube' in link:
        links.remove(link)
      elif 'alumni' in link:
        links.remove(link)
      elif 'drive.google' in link:
        links.remove(link)
      elif('.instagram.' in link or 'google' in link or 'twitter' in link or 'facebook' in link):
        links.remove(link)
      elif 'rds3' in link or 'login' in link:
        links.remove(link)
      else:
        pass
    new_length = len(links)

  return links


format(links)

print(len(links))
#for link in links:
#  print((link))

133


Total formatted links: 

In [ ]:
len(links)

133

fetching links from the list of links (links from sub pages) | 4 mins execution time

In [ ]:
links_major = [] 

total = len(links)
i = 1
for link in links:
  index = link
  document = requests.get(index)
  soup = BeautifulSoup(document.text, "html.parser")
  print("fetching links:  %d of 133" %i)
  for l in soup.findAll('a'):
    var = l.get('href')
    if(var == '#' or var is None):
      continue
    else:
      if('http' in var):
        #print(var)
        links_major.append(var)
      else:
        var = index + var
        links_major.append(var)
        #print(var)

print("***  Done.  ***")
links_major += links # copy contents

fetching links:  0 of 133
fetching links:  1 of 133
fetching links:  2 of 133
fetching links:  3 of 133
fetching links:  4 of 133
fetching links:  5 of 133
fetching links:  6 of 133
fetching links:  7 of 133
fetching links:  8 of 133
fetching links:  9 of 133
fetching links:  10 of 133
fetching links:  11 of 133
fetching links:  12 of 133
fetching links:  13 of 133
fetching links:  14 of 133
fetching links:  15 of 133
fetching links:  16 of 133
fetching links:  17 of 133
fetching links:  18 of 133
fetching links:  19 of 133
fetching links:  20 of 133
fetching links:  21 of 133
fetching links:  22 of 133
fetching links:  23 of 133
fetching links:  24 of 133
fetching links:  25 of 133
fetching links:  26 of 133
fetching links:  27 of 133
fetching links:  28 of 133
fetching links:  29 of 133
fetching links:  30 of 133
fetching links:  31 of 133
fetching links:  32 of 133
fetching links:  33 of 133
fetching links:  34 of 133
fetching links:  35 of 133
fetching links:  36 of 133
fetching li

remove duplicates and format

In [ ]:
links_major = list(dict.fromkeys(links_major))

format(links_major)

print(len(links_major))

8008


In [ ]:
title_list = []
text_list = []

def fetch(link):
  content = requests.get(link)
  #text_file = open("data.txt", "a")
  title_df = ""
  document = ""
  soup = BeautifulSoup(content.text, "html.parser")

  try: #check
    title = soup.find('title').string
    #text_file.write(str(title))
    title_df = title
    print(title)
  except:
    print("*Title does not exist for this page. *")

  try:
    for var in soup.findAll('h1'):
      #text_file.write(str(var.text.strip()))
      print(var.string)
      #document += (var.string)

    for var in soup.findAll('h2'):
      #text_file.write(str(var.text.strip()))
      print(var.text.strip())
      #document += str(var.text.strip())

    for var in soup.findAll('h3'):
      #text_file.write(str(var.text.strip()))
      print(var.text.strip())
      #document += str(var.text.strip())

    for var in soup.findAll('h6'):
      #text_file.write(str(var.text.strip()))
      print(var.text.strip())
      #document += str(var.text.strip())
  except:
    print("error in text extraction for h elements")
    pass

  try:
    for var in soup.findAll('p'):
      #text_file.write(str(var.text.strip()))
      print(var.string)
      document += var.string
  except:
      print("error in text extraction")
      pass

  divs = soup.find_all('div')

  for div in divs:
    try:
      if len(div.string) > 0:
        print(div.string)
    except:
      continue

  title_list.append(title_df)
  text_list.append(document)
  #text_file.close()

fetch contents from links

In [ ]:
#dataframe = pd.DataFrame(columns=['Title', 'Text'])
dataframe = pd.DataFrame(columns=['Title', 'Text'])

for link in links:
  fetch(link)

Contact  | North South University
Contact
Location Map
None
error in text extraction
Center of Excellence in Higher Education 
The First Private University in Bangladesh
Catalogs | North South University
Catalogs
None
error in text extraction
Center of Excellence in Higher Education 
The First Private University in Bangladesh
Journals | North South University
Journals
NSBR Copy Right Agreement..
NSBR Administration, Policy, and Guideline..
NSBR Publication Ethics and Malpractices Statement for NSBR..
 
 
 
None
error in text extraction
Center of Excellence in Higher Education 
The First Private University in Bangladesh
Newsletters | North South University
Newsletters
None
error in text extraction
Center of Excellence in Higher Education 
The First Private University in Bangladesh
Home - NSU Library
Bangabandhu & Liberation War Corner

Latest News & Events
Workshop Program on SCOPUS for the Senior Faculty Members of NSU
Information and Broadcasting Minister Dr Hasan Mahmud visited NSU L

# Preparing the dataset

In [ ]:
#title = pd.DataFrame(title_list, columns=["Title"])
text = pd.DataFrame(text_list, columns=["Text"])

#temp = pd.concat([title, text], axis = 1)

dataframe = pd.concat([dataframe, text], axis = 0)

In [ ]:
print(dataframe.head())

  Title                                               Text
0   NaN                                                   
1   NaN                                                   
2   NaN  NSBR Copy Right Agreement..NSBR Administration...
3   NaN                                                   
4   NaN  Total 2,178 title books collection are availab...


In [ ]:
print(len(title_list))
print(len(text_list))

133
133


In [ ]:
dataframe.dropna(inplace=True)
dataframe.drop_duplicates(inplace=True)
dataframe.to_csv(r'dataframe.csv', index=False)

remove blank strings from list

In [ ]:
print(len(text_list))
# remove
while("" in text_list) :
    text_list.remove("")

print(len(text_list))

sep

In [230]:
separated_doc = []
# txt is a dummy list for debug :) , 0-th index consists of 166 words.
#txt = ['But I must explain to you how all this mistaken idea of denouncing pleasure and praising pain was born and I will give you a complete account of the system, and expound the actual teachings of the great explorer of the truth, the master-builder of human happiness. No one rejects, dislikes, or avoids pleasure itself, because it is pleasure, but because those who do not know how to pursue pleasure rationally encounter consequences that are extremely painful. Nor again is there anyone who loves or pursues or desires to obtain pain of itself, because it is pain, but because occasionally circumstances occur in which toil and pain can procure him some great pleasure. To take a trivial example, which of us ever undertakes laborious physical exercise, except to obtain some advantage from it? But who has any right to find fault with a man who chooses to enjoy a pleasure that has no annoying consequences, or one who avoids a pain that produces no resultant pleasure', 'abc', " ", "", "   "]
print("before splitting: %d" %(len(text_list)))
for s in text_list:
  #res = len(s.split(" "))
  counter = 0
  words = s.split()
  for i in range(0, len(words), 100):
    chunked_words = words[i : i+100]
    recombined_words = " ".join(chunked_words)
    recombined_words = recombined_words.strip() # strip trailing white spaces
    #separated_doc.append(recombined_words)
    if len(words) == 0:
      counter+=1
      continue
    else:
      separated_doc.append(recombined_words)

print("after splitting: %d" %(len(separated_doc)))
print("text with blank spaces in the last: %d" %(counter))

before splitting: 59
after splitting: 96
text with blank spaces in the last: 0


In [ ]:
for s in separated_doc:
  print(len(s.split()))

In [232]:
#title = pd.DataFrame(title_list, columns=["Title"])
text = pd.DataFrame(separated_doc, columns=["Text"])

#temp = pd.concat([title, text], axis = 1)
dataframe = dataframe.iloc[0:0]
dataframe = pd.concat([dataframe, text], axis = 0)

In [233]:
print(dataframe.head())
#dataframe.dropna(inplace=True)
dataframe.drop_duplicates(inplace=True)
dataframe.to_csv(r'dataframe_new.csv', index=True)

  Title                                               Text
0   NaN  NSBR Copy Right Agreement..NSBR Administration...
1   NaN  Total 2,178 title books collection are availab...
2   NaN  North South University (NSU), the first privat...
3   NaN  education in the private sector. Their dedicat...
4   NaN  February 1993 and started its journey in a ver...
